In [3]:
import crawler
import importlib
importlib.reload(crawler)

<module 'crawler' from '/opt/ccclub/sola/crawler/crawler.py'>

In [2]:
from crawler import Crawler                
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json

In [4]:
# 初始chrome
crawler_ccclub = Crawler(remote=True, diff_container=True)
driver = crawler_ccclub.configure_driver()

In [5]:
url_login = "https://docs.google.com/spreadsheets/u/0/d/1fQh2oRUWNJWkjiHoijqGZ-xqo1x5i_v8196LdXJGK1w/htmlview"
driver.get(url_login)
time.sleep(4)


In [20]:
html_content = driver.page_source
import pandas as pd
df =pd.read_html(html_content)

/tmp/ipykernel_74556/2672288770.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df =pd.read_html(html_content)


In [34]:
df_table = df[0]
new_columns = df_table.iloc[0]
df_table.columns = new_columns
df_table = df_table[2:]
print(df_table.columns)
columns_to_keep = ['公司名稱', '職務', '職級','相關年資','現職年資','底薪',
                   'Bonus(月)','總年薪(萬) 分紅+年終+底薪','每日平均工時',
                   '每月加班','加班頻率', '爽度(1~5) 5最爽','Loading',
                   '心得 是否推薦 面試相關....etc','地點', 
                   '股票總價值以及 Vesting Schedule',]  # 替换为你想保留的列名
df_table = df_table[columns_to_keep]
df_table

Index([                       1.0,                     '公司名稱',
                             '職務',                        nan,
                             '職級',                     '相關年資',
                           '現職年資',                       '底薪',
                       'Bonus(月)',          '總年薪(萬) 分紅+年終+底薪',
                         '每日平均工時',                     '每月加班',
                           '加班頻率',              '爽度(1~5) 5最爽',
                        'Loading',      '心得 是否推薦 面試相關....etc',
                             '地點', '股票總價值以及 Vesting Schedule',
                              nan,                        nan,
                              nan,                        nan,
                              nan,                        nan,
                              nan],
      dtype='object', name=0)


KeyError: "['總年薪(萬) 分紅+年終+底薪每日平均工時'] not in index"

In [6]:
# step 01 - 處理 login
url_login = "https://www.ccclub.io/login"

driver.get(url_login)
time.sleep(2)
input_email = driver.find_element(By.CSS_SELECTOR, 'input[type=text]')
input_password = driver.find_element(By.CSS_SELECTOR, 'input[type=password]')
button_login = driver.find_element(By.CLASS_NAME, 'loginBtn')

email = "XXXXXXXXXXX@gmail.com"
password = "ctC6CQZgaaaqf"

input_email.send_keys(email)
time.sleep(2)
input_password.send_keys(password)
time.sleep(2)
button_login.click()

In [ ]:
# step 02 -進入achievement
url="https://www.ccclub.io/achievement"
driver.get(url)
print("Chrome driver is available.")
time.sleep(2)

In [ ]:
def scroll_to_bottom():
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            # print("已經到底")
            break
        last_height = new_height
    time.sleep(2)

scroll_to_bottom()

In [ ]:
def click_page(page:int):
    try:
        # 显式等待，直到class为'pagination-link'且text为'2'的元素出现
        element = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, f"//a[@class='pagination-link' and text()='{str(page)}']"))
        )
        # 输出找到的元素
        # print("Element found:", element)
        # print("Element text:", element.text)
        element.click()
        # print("Element clicked successfully.")
        time.sleep(5)
    
    except Exception as e:
        print(f"An error occurred: {e}")


In [ ]:
def fetch_data():
    gallery_items = []
    
    elements = driver.find_elements(By.CLASS_NAME, 'achievement__gallery-item-body')
    
    for idx, item in enumerate(elements):
        item_data = {}
        # print(f"Gallery Item {idx + 1}:", end=' | ')
        item_name = item.find_element(By.CLASS_NAME, 'achievement__gallery-item-name')
        item_author = item.find_element(By.CLASS_NAME, 'achievement__gallery-item-author')
        item_tags = item.find_element(By.CLASS_NAME, 'achievement__gallery-item-tags')
        item_actions = item.find_elements(By.CLASS_NAME, 'achievement__gallery-item-actions')
        
        item_data['Name']=item_name.text.strip()
        item_data['Author']=item_author.text.strip().split('、')
        item_data['tags']=[tag.lstrip('#') for tag in item_tags.text.strip().split()]
    
        for item_action in item_actions:
            links = item_action.find_elements(By.CLASS_NAME, 'achievement__gallery-item-action-link')
            for link in links:
                # print(link.text)
                if "看發表影片" in link.text.strip():
                    # print(f"link-video:{link.get_attribute('href')}")
                    item_data['link-video'] = link.get_attribute('href')
                if "下載簡報"in link.text.strip():
                    # print(f"link-pdf:{link.get_attribute('href')}")
                    item_data['link-pdf'] = link.get_attribute('href')
                if "Github" in link.text.strip():
                    # print(f"link-github:{link.get_attribute('href')}")
                    item_data['link-github'] = link.get_attribute('href')
                    
        gallery_items.append(item_data)
    
    return gallery_items

In [ ]:
def get_max_page():
    elements = driver.find_elements(By.CLASS_NAME, 'pagination-link')
    max_page = 0
    for element in elements:
        if element.text.strip() != "":
            page = int(element.text)
            max_page = max(page, max_page)
    return max_page

In [ ]:
def get_current_page():
    element = driver.find_element(By.XPATH, "//a[contains(@class, 'pagination-link') and contains(@class, 'is-current')]")
    page = int(element.text)
    return page

In [ ]:
max_page = get_max_page()
page = 1
all_items = []

for _ in range(max_page):
    time.sleep(2)
    current = get_current_page()
    if page == current:
        scroll_to_bottom()
        print(f"現在正在第{page}頁")
        page_data = fetch_data()
        all_items.extend(page_data)
        time.sleep(2)
    # 進入下一頁
    if page <= max_page:
        page +=1
        click_page(page)

In [ ]:
driver.quit()

In [ ]:
# 存成json檔案
import json
json_filename = 'ccclub_achievement.json'

def save_json(all_items, json_filename):
    json_data = json.dumps(all_items, ensure_ascii=False, indent=4)
    
    with open(json_filename, 'w' ,encoding='utf-8') as json_file:
        json_file.write(json_data)
        print(f"Save json file: {json_filename}")

save_json(all_items, json_filename)

In [3]:
# 讀出json檔案 轉成excle
import pandas as pd

def json_to_excel(json_filename, excel_filename):
    with open(json_filename, 'r', encoding='utf-8') as file:
        file_content = json.load(file)
        # 處理json檔案(適合excel)
        df = pd.json_normalize(file_content)
        df.to_excel(excel_filename, index=False)
        print(len(df))
    print(f"Transfer from JSON into Excel: {excel_filename}")

json_filename = 'ccclub_achievement.json'
excel_filename = 'ccclub_achievement.xlsx'
json_to_excel(json_filename, excel_filename)

310
Transfer from JSON into Excel: ccclub_achievement.xlsx


In [30]:
# 將link欄位變成超連結(text顯示)

import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.worksheet.hyperlink import Hyperlink

# 使用 openpyxl 加載生成的 Excel 文件
wb = load_workbook('project.xlsx')
ws = wb.active

# 310個專案

def fix_link(display_text, url_index, target_index):
    # 設置超連結
    for row in ws.iter_rows(min_row=2, max_col=9, max_row=311):
        
        # title
        ws[1][target_index].value = display_text
        
        url_pdf_cell = row[url_index] # D column
        url = url_pdf_cell.value
        if url and url.strip():
            display_cell = row[target_index] # G column
            # 設置顯示文本為超連結
            display_cell.hyperlink = Hyperlink(ref=display_cell.coordinate, target=url)
            display_cell.value = display_text
            display_cell.style = "Hyperlink"

display_text = "pdf_link"
url_index = 3
target_index = 6
fix_link(display_text, url_index, target_index)

display_text = "pdf_video"
url_index = 4
target_index = 7
fix_link(display_text, url_index, target_index)

display_text = "pdf_github"
url_index = 5
target_index = 8
fix_link(display_text, url_index, target_index)

# 需要隐藏的列索引
hidden_columns = [3,4,5]  #要隐藏第3列

# 设置隐藏列
for col_idx in hidden_columns:
    ws.column_dimensions[ws.cell(row=1, column=col_idx + 1).column_letter].hidden = True


# 保存修改過的 Excel 文件
wb.save('project.xlsx')
